In [ ]:
import argparse
import torch
import torch.nn as nn
import numpy as np
from sklearn.cluster import KMeans
import torch.optim as optim
from sklearn import metrics
from tqdm import tqdm

from util import load_data, load_graph
from models.udci import U_DCI
from models.clf_model import Classifier

In [ ]:
def setup_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True


In [ ]:
def finetune(args, model_pretrain, device, test_graph, feats_num):
    model = Classifier(args.num_layers, args.num_mlp_layers, feats_num, args.hidden_dim, args.final_dropout, args.neighbor_pooling_type, device).to(device)
    
    # replace the encoder in joint model with the pre-trained encoder，把外面训练好的CDI导进来，这个视角encoder吗
    pretrained_dict = model_pretrain.state_dict()   # state_dict()是pytorch里调用所有参数信息的函数，这里是把DCI的参数信息存起来
    model_dict = model.state_dict() #这里就是吧Classifier的参数信息存起来
    pretrained_dict =  {k: v for k, v in pretrained_dict.items() if k in model_dict}    # 只留下Classifier里有的字段的DCI的参数信息，为了防止下一步错叭
    model_dict.update(pretrained_dict)  # 这两步就是把Classifier的参数信息更新成DCI的参数信息
    model.load_state_dict(model_dict)

    optimizer = optim.Adam(model.parameters(), lr=args.lr)  # 优化器的意思应该是反向传播的那个，不要纠结细节上次学过了虽然我知道你忘了
    
    criterion_tune = nn.BCEWithLogitsLoss() # 二分类的交叉熵损失函数

    res = []
    train_idx = test_graph[2]
    node_train = test_graph[-1][train_idx, 0].astype('int')
    label_train = torch.FloatTensor(test_graph[-1][train_idx, 1]).to(device)
    for _ in range(1, args.finetune_epochs+1):  # 这一段应该就是在训练
        model.train()
        output = model(test_graph[0], test_graph[1])
        loss = criterion_tune(output[node_train], torch.reshape(label_train, (-1, 1)))
        
        #backprop
        if optimizer is not None:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        # testing
        model.eval()    # 设成eval模式
        auc = evaluate(model, test_graph)
        res.append(auc)

    return np.max(res)  # 返回最大的auc

In [1]:
def evaluate(model, test_graph):
    output = model(test_graph[0], test_graph[1])    #预测结果，其中0和1分别是adj和features，顺序记得确认一下
    pred = sig(output.detach().cpu())   # 用激活函数压缩一下然后结果传到cpu上
    test_idx = test_graph[3]    # 这个idx具体指什么要看model，我猜是abnormal data的idx
    
    labels = test_graph[-1] # 那么labels我猜就是abnormal data的label
    pred = pred[labels[test_idx, 0].astype('int')].numpy()  # 不知道了这里回头再来看吧
    target = labels[test_idx, 1]
    
    false_positive_rate, true_positive_rate, _ = metrics.roc_curve(target, pred, pos_label=1)
    auc = metrics.auc(false_positive_rate, true_positive_rate)  # auc越接近1越好

    return auc

In [ ]:
# set parameters for U_DCI
config_udci = {
    'dataset': 'wiki',              # name of dataset
    'device': 0,                    # which gpu to use if any
    'epochs': 50,                   # number of epochs in pre-training stage
    'num_cluster': 2,               # number of clusters in pre-training stage
    'recluster_interval': 10,       # recluster interval in pre-training stage
    'finetune_epochs': 100,         # number of epochs in finetune stage
    'num_folds': 10,                # number of folds in cross validation of finetune stage
    'final_dropout': 0.5,           # dropout rate used in finetune stage
    'lr': 0.01,                     # learning rate used in optimizer
    'training_scheme': 'decoupled'  # 'decoupled' or 'joint'
    
}

In [ ]:
# set hyperparameters for embedding module

# U_GCN
config_emb = {
    'in_features': 64,      # number of initial feature dims, used in GraphAttention
    'out_features': 16,     # number of feature dims generated by one attention head, used in GraphAttention
    'alpha': 0.2,           # slope of leakyReLU, used in GraphAttention
    'final_features': 16,   # number of feature dims integrated out of all attention heads, used in GraphAttention
    'dropout': 0.6,         # customized dropout rate, used in GraphAttention & GAT
    'nheads': 8,            # number of attetnion heads, used in GAT
    'hidden_dim': 16,       # number of hidden dims, used in Discriminator
}

# GIN
# confit_emb = {}

In [ ]:
sig = torch.nn.Sigmoid()
setup_seed(0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

edge_index, feats, split_idx, label, nb_nodes = load_data(config_udci['dataset'], config_udci['num_folds'])
input_dim = feats.shape[1]

kmeans = KMeans(n_clusters=config_udci['num_cluster'], random_state=0).fit(feats)
ss_label = kmeans.labels_
cluster_info = [list(np.where(ss_label==i)[0]) for i in range(config_udci['num_cluster'])]

idx = np.random.permutation(nb_nodes)
shuf_feats = feats[idx, :]